In [1]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans,Dataset,accuracy,Reader
from surprise.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import sklearn

In [2]:
# loading all the csv files
ratings = pd.read_csv('C:/Users/Srishti/Desktop/Data Science/Machine Learning/Assignments/Assignment3/ml-latest-small/ratings.csv')
movies = pd.read_csv('C:/Users/Srishti/Desktop/Data Science/Machine Learning/Assignments/Assignment3/ml-latest-small/movies.csv')
links = pd.read_csv('C:/Users/Srishti/Desktop/Data Science/Machine Learning/Assignments/Assignment3/ml-latest-small/links.csv')
tags = pd.read_csv('C:/Users/Srishti/Desktop/Data Science/Machine Learning/Assignments/Assignment3/ml-latest-small/tags.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
#get the column names and shape of ratings dataframe
ratings.columns,ratings.shape

(Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object'),
 (100836, 4))

In [4]:
#get the column names and shape of movies dataframe
movies.columns,movies.shape

(Index(['movieId', 'title', 'genres'], dtype='object'), (9742, 3))

In [5]:
#get the column names and shape of links dataframe
links.columns,links.shape

(Index(['movieId', 'imdbId', 'tmdbId'], dtype='object'), (9742, 3))

In [6]:
#get the column names and shape of tags dataframe
tags.columns,tags.shape

(Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object'), (3683, 4))

#### 1.Create recommender system using ratings.csv file. 

In [7]:
#get the range of rating
ratings.rating.value_counts()  #the range is from 0 to 5

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [8]:
ratings_pivot=(ratings.pivot(index='movieId',columns='userId',
                      values='rating').fillna(0))
ratings_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
ratings_matrix=csr_matrix(ratings_pivot.values)
ratings_matrix

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100836 stored elements in Compressed Sparse Row format>

In [10]:
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(ratings_matrix) 

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [11]:
query_index1=int(input('Enter movieId: ')) #1st movie
distances,indices=model_knn.kneighbors(
    ratings_pivot.loc[query_index1,:].values.reshape(1,-1),
    n_neighbors=11)

Enter movieId: 23


In [12]:
#Display the distances from shorter to longer:
distances

array([[1.11022302e-16, 5.73607211e-01, 5.97377021e-01, 6.01651762e-01,
        6.19442796e-01, 6.25834261e-01, 6.29375342e-01, 6.36918972e-01,
        6.37758302e-01, 6.49657702e-01, 6.52102870e-01]])

In [13]:
#Display the indices(movie positions) according to the respective distance found above:
indices

array([[  22,  146,  283,  223,  238, 4933,  421,  159,    7, 2089, 3647]],
      dtype=int64)

In [14]:
print('Recommendations for movieId {0}\n'.format(ratings_pivot.index[query_index1]))
for i in indices.flatten()[1:]:
    print('For',[i],'th position, the movieId is:',ratings_pivot.index[i])

Recommendations for movieId 24

For [146] th position, the movieId is: 174
For [283] th position, the movieId is: 325
For [223] th position, the movieId is: 259
For [238] th position, the movieId is: 276
For [4933] th position, the movieId is: 7449
For [421] th position, the movieId is: 484
For [159] th position, the movieId is: 188
For [7] th position, the movieId is: 8
For [2089] th position, the movieId is: 2782
For [3647] th position, the movieId is: 5016


#### 2.Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [15]:
#reader class to mention the rating scale from 0.5 to 5
reader=Reader(rating_scale=(0.5,5))

#creating a dataframe which is to be given to the train_test_split for spliting the data
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

In [16]:
#spliting the data into trainset and testset
[trainset,testset]=train_test_split(data,test_size=0.15,shuffle=True)

recommend=KNNWithMeans(k=9,sim_options={'name':'cosine','user_based':True})
recommend.fit(data.build_full_trainset()) #using full training set
test_pred=recommend.test(testset)
RMSE=accuracy.rmse(test_pred)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6888


In [17]:
# Taking the UserID and MovieId as a user input from the keyboard and displaying the predicted rating of the user
user_id = int(input("Enter userId: "))
movie_id = int(input("Enter movieId: "))

#predicting the rating as per userId and movieId taken from user
pre=recommend.predict(user_id,movie_id)
print("Predicted MovieId {} Rating for the UserID {} is %.2f".format(movie_id,user_id)% pre.est)

Enter userId: 20
Enter movieId: 2
Predicted MovieId 2 Rating for the UserID 20 is 3.43


#### 3.Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5

In [18]:
ratings_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
user_id=int(input('Enter userId:'))
counter=0
list1=[]
list2=[]
for i in ratings_pivot[user_id]:
    if i==0:
        prediction=recommend.predict(user_id,ratings_pivot.loc[:,user_id].keys()[counter])
        if prediction.est>=2.5:
            list1.append(prediction.iid)
            list2.append(prediction.est)
    counter+=1
dataframe=pd.DataFrame()
dataframe['movieId']=list1
dataframe['rating']=list2
top_10_movies=dataframe.sort_values(by='rating',ascending=False).head(10)
top_10_movies

Enter userId:25


,movieId,rating
0,1,5.0
4991,7932,5.0
4989,7926,5.0
4988,7925,5.0
4983,7899,5.0
4981,7895,5.0
4980,7894,5.0
4977,7888,5.0
4970,7844,5.0
4969,7843,5.0


In [20]:
merged_movie=top_10_movies.merge(movies,on='movieId')
new_merged_df=merged_movie.drop(['genres'],axis=1)
new_merged_df

,movieId,rating,title
0,1,5.0,Toy Story (1995)
1,7932,5.0,Dark Days (2000)
2,7926,5.0,High and Low (Tengoku to jigoku) (1963)
3,7925,5.0,"Hidden Fortress, The (Kakushi-toride no san-ak..."
4,7899,5.0,Master of the Flying Guillotine (Du bi quan wa...
5,7895,5.0,Bring Me the Head of Alfredo Garcia (1974)
6,7894,5.0,"Duck, You Sucker (1971)"
7,7888,5.0,How to Succeed in Business Without Really Tryi...
8,7844,5.0,"Legend, The (Legend of Fong Sai-Yuk, The) (Fon..."
9,7843,5.0,Lammbock (2001)


#### 4.Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.

In [21]:
dataframe1=movies.merge(ratings)
average_ratings_bf_pred=dataframe1.groupby('movieId')['rating'].mean().reset_index()
average_ratings_bf_pred.columns=['movieId','Average_rating_bf_pred']
average_ratings_bf_pred.head()

,movieId,Average_rating_bf_pred
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [22]:
dataframe2=new_merged_df.merge(movies).merge(links).merge(tags,on='movieId',how='left')
dataframe3=dataframe2.drop(['tmdbId','userId','timestamp'],axis=1)
final_dataframe=dataframe3.merge(average_ratings_bf_pred)
final_dataframe

,movieId,rating,title,genres,imdbId,tag,Average_rating_bf_pred
0,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,pixar,3.920930
1,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,pixar,3.920930
2,1,5.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,fun,3.920930
3,7932,5.0,Dark Days (2000),Documentary,235327,Amtrak,3.666667
4,7932,5.0,Dark Days (2000),Documentary,235327,Homeless,3.666667
5,7932,5.0,Dark Days (2000),Documentary,235327,independent,3.666667
6,7932,5.0,Dark Days (2000),Documentary,235327,New York,3.666667
7,7932,5.0,Dark Days (2000),Documentary,235327,Sundance award winner,3.666667
8,7932,5.0,Dark Days (2000),Documentary,235327,Train,3.666667
9,7926,5.0,High and Low (Tengoku to jigoku) (1963),Crime|Drama|Film-Noir|Thriller,57565,In Netflix queue,4.000000
